## Model/training pipelines:
### Visual Transformer Training Pipeline for Alzheimer's Classification

## 1. Load Dataset
- Load training and test datasets using `ImageFolder`.
- Use `DataLoader` to handle batching, shuffling, and parallel data loading.

## 2. Model Initialization
- Initialize a pretrained Vision Transformer (ViT) model.
- Modify the model for binary classification (Alzheimer's Disease vs. No Alzheimer's).

## 3. Define Loss and Optimizer
- Use Cross-Entropy Loss for classification tasks.
- Choose an optimizer (e.g., Adam the GOAT) to update model weights during training.

## 4. Training Loop
- For each epoch:
    - Feed batches of images through the model.
    - Compute the loss and perform backpropagation to adjust the weights.
    - Track training loss across batches.

## 5. Evaluation Loop
- After training, evaluate the model using the test dataset.
- Calculate accuracy and other relevant metrics to measure model performance.

## 6. Fine-tuning and Optimization
- Optionally implement techniques like learning rate scheduling, early stopping, and model checkpointing to improve training efficiency and model performance.


In [1]:
import torch
import torchvision


In [5]:

train_dir = "./ADNI/AD_NC/train"
test_dir = "./ADNI/AD_NC/test"


train_data = torchvision.datasets.ImageFolder(root= train_dir)
test_data = torchvision.datasets.ImageFolder(root= test_dir)

In [6]:
### Exploring the image data and stuff
len(train_data), train_data.classes, train_data[0], train_data.class_to_idx

### Size is 256 x 240, RGB so 3 channels, and the 2nd column is the label. I believe all 0 labels are the first half of the training set
## So make sure to shuffle!!

(21520,
 ['AD', 'NC'],
 (<PIL.Image.Image image mode=RGB size=256x240>, 0),
 {'AD': 0, 'NC': 1})

### Set up loaders
- Set up loaders so that we can shuffle data, do batching, and load 

In [ ]:
from torch.utils.data import DataLoader

# Define batch size
batch_size = 32  # You can adjust this based on your GPU memory
num_workers = 4

# Creating the DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# Shuffle not needed for test because we aren't training on it. Shuffle is mainly used because, imagine if you had all your 0 classifications in the beginning and the dataloader ate this. You're feeding the NN hundreds of entire "0s" that it's going to backprop in this direction, instead of an 'unbiased sample'. This can affect how well it learns. Shuffling helps better convergence, and generalisation.

test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
